In [1]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

os.chdir('../')

In [2]:
df = pd.read_csv('data/processed/preprocessed.csv')

### Rule-based метод

Так как в тексте могут быть явные указатели, при этом ответ почти всегда следует одному шаблону, запишем в answer уже явные ответы.  
Может стоило объеденить классы ещё больше, но тогда будут слишком абстрактные ответы.

Вообще кажется что зачем вообще модель, если можно сделать просто кучу правил, они ведь работают на шаблонных фразах. Но модель можно применять для текстов с похожими смыслами, перефразированиями, ошибками в орфографии, которые не справляются spellcheckers

In [3]:
import re

# Функции чтобы искать именно слова, а не подстроки
# Хотя бы одно слово
def has_any_words(words, text):
    pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    return bool(re.search(pattern, text, flags=re.IGNORECASE))

# Хотя бы все слова
def has_all_words(words, text):
    for word in words:
        pattern = r'\b' + re.escape(word) + r'\b'
        if not re.search(pattern, text, flags=re.IGNORECASE):
            return False
    return True

# Тут я пытался сделать через список, функции, сочетание с lambda этих способов, 
# но ничего не работало из-за логики определения удовлетворено/частично удовлетворено/отказ и без вложенных if elif это не сделать
# Пока что это тяжело читать и расширять, но это делается легко и быстро. То есть это не в моего опыта сделать это лучше
# Ответы не точные как если бы сгенерировала LLM, много объеденений, но это быстрее и бесплатно
def extract_status(text):
    if (has_all_words(['подлежащее удовлетворение'], text)):
        if (has_any_words(['не_подлежащее', 'не_удовлетворение'], text)): # В примерах могут быть удовлетворения и не удовлетворения одновременно
            return 'Частично удовлетворено'
        return 'Удовлетворено'
    
    elif (has_any_words(['не_подлежащее', 'не_удовлетворение'], text)):
        if (has_all_words(['подлежащее', 'удовлетворение'], text)):
            return 'Частично удовлетворено'
        return 'Отказ'
    
    # elif ((has_all_words(['окончить', 'производство'], text)) # переместил в отказ, так как слишком маленький класс (9 примеров), а так лучше бы оставить
    #       or (has_all_words(['банкротство', 'внесудебный'], text))):
    #     return 'Производство окончено'

    elif (has_all_words(['срок', 'мера'], text)):
        return 'Частично удовлетворено'

    elif (has_all_words(['обратить', 'взыскание'], text)
          or has_all_words(['обращение', 'взыскание',], text)
          and not has_all_words(['не_иметься'], text)):
        return 'Взыскание обращено'
    
    elif (has_any_words(['действие', 'рег', 'регистрационный'], text)
          and has_all_words(['запрет'], text)):
        return 'Запрет действий'

    elif has_all_words(['вынести', 'постановление'], text):
        return 'Постановление вынесено'
    
    elif has_all_words(['направить', 'запрос'], text):
        return 'Запрос направлен'
    
    elif ((has_all_words(['обращение', 'рассмотреть'], text)
           or has_all_words(['обращение', 'ответ'], text))
          or (has_all_words(['производство', 'исполнительный', 'госуслуга'], text))
          or ((has_any_words(['портал', 'подпись'], text) and has_any_words(['информационный', 'электронный'], text)))):
        return 'Обращение рассмотрено'
    
    elif has_all_words(['исполнительный', 'производство', 'проверка'], text) and has_any_words(['жалоба', 'ходатайство', 'заявление'], text):
        return 'Заявления и жалобы рассматриваются'
    
    elif has_all_words(['исполнительный', 'производство', 'исполнительный', 'документ'], text):
        return 'Возбуждено исполнительное производство'
    
    elif has_all_words(['исполнительный', 'розыск', 'должник'], text):
        return 'Объявлен исполнительный розыск'
    
    elif ((has_all_words(['мера', 'принудительный'], text))
          or has_all_words(['исполнительный', 'производство', 'регистрировать'], text)):
        return 'Применены меры для исполнения'
    
    elif ((has_all_words(['невозможно', 'установить'], text)
           or has_all_words(['не_представиться', 'лицо'], text)
           or has_all_words(['не_установить', 'должник'], text))
          or (has_all_words(['не_являться', 'рассмотрение'], text))
          or (has_all_words(['жалоба', 'бездействие', 'пристав', 'правомерный'], text))
          or (has_all_words(['окончить', 'производство'], text)) or (has_all_words(['банкротство', 'внесудебный'], text))):
        return 'Отказ'
    
    else:
        return None

In [4]:
df['answer'] = df['lemmatized_text'].apply(extract_status)

print(df['answer'].value_counts())
print("\nВсего обработано строк:", df['answer'].value_counts().sum())

answer
Отказ                                     217
Частично удовлетворено                    138
Обращение рассмотрено                     130
Взыскание обращено                        106
Запрос направлен                           80
Возбуждено исполнительное производство     76
Постановление вынесено                     37
Удовлетворено                              27
Заявления и жалобы рассматриваются         26
Объявлен исполнительный розыск             20
Применены меры для исполнения              19
Запрет действий                            14
Name: count, dtype: int64

Всего обработано строк: 890


Cтоит убрать классы меньше чем с 10 примерами, так как они позже будут мешать обучению модели.  
Если же переименовать такие классы на просто "другие", то модель может классифицировать многие тексты, как "другие", что не очень то является ответом на заявление

### Итог

Таким образом осталось ~треть данных из 1000 строк  
Остались только не входящие в правила строки и, возможно, неправильно обработанные

Пока разметка классов идёт по первым встреченным словам.  
Проблема в том, что одна строка может иметь много классов, что переходит в многолейбловую классификацию, что не оставляет самые распространённые методы по типу SVM, Log Reg, K-means, а требует что-то похожее на MultiOutputClassifier.  
Также среди размеченных данных могут быть такие классы, которые встречаются только в числе других классов, а не одиночно, что значит не хватает размера датасета и модель просто не сможет правильно классифицировать.  

In [5]:
df.to_csv('data/processed/rule_classified.csv', index=False)